<a href="https://colab.research.google.com/github/pq2312/Analyzing-Stock-Performance-During-the-Covid-Crisis-2018-2022-/blob/main/Analyzing_Stock_Performance_During_the_Covid_Crisis_(2018_2022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Stock Performance During the Covid Crisis (2018-2022)


**EXECUTIVE SUMMARY**

**Analyzing Stock Performance During the Covid Crisis (2018-2022)**

This analysis examines how certain stocks fared during the Covid-19 pandemic (2018-2022) using the Fama-French 3-factor (FF3) regression model. The FF3 model helps assess risk exposures by considering market risk, size risk, and value risk.

***Winners and Losers***

The FF3 model identified some stocks that outperformed expectations (positive alpha) and others that underperformed (negative alpha).

* Top Performers: Leading the pack were KOSS (home entertainment products), GME (gaming), and UONE (multimedia). These industries likely experienced a boom due to pandemic-driven changes in consumer behavior, such as increased demand for home entertainment products and gaming during lockdowns.

* Underperformers:  VIVE (women's intimate health), TNXP (treatments for psychiatric and neurological conditions), and INPX (indoor positioning solutions) fell short of expectations. The pandemic might have disrupted their industries, caused revenue generation challenges, or introduced uncertainties around future business prospects due to economic instability and reduced consumer spending.

### Import libraries and packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Upload Monthly Stock Return Data for 2018-2022
* CSV file contains monthly stock returns for many stocks for the period 2018-2022

In [ ]:
ret_data = pd.read_csv('CRSP-2018-2022.csv')


### Examine variables in dataframe

In [ ]:
ret_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485387 entries, 0 to 485386
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   PERMNO  485387 non-null  int64  
 1   date    485387 non-null  object 
 2   TICKER  485382 non-null  object 
 3   CUSIP   485387 non-null  object 
 4   RET     485387 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 18.5+ MB


###Print out header of dataframe for complete file

In [ ]:
ret_data.head()

,PERMNO,date,TICKER,CUSIP,RET
0,10026,2018-01-31,JJSF,46603210,-0.088191
1,10026,2018-02-28,JJSF,46603210,-0.029688
2,10026,2018-03-29,JJSF,46603210,0.019951
3,10026,2018-04-30,JJSF,46603210,0.006224
4,10026,2018-05-31,JJSF,46603210,0.030638


In [ ]:
#COUNT VALUE
ret_data['TICKER'].value_counts()

TICKER
BF      120
BIO     120
HVT     120
HEI     120
BRK     120
       ... 
NAMS      1
DRS       1
HVBT      1
TORM      1
STRP      1
Name: count, Length: 12482, dtype: int64

###Upload Fama-French monthly risk factor data


*   CSV file contains data on monthly market risk, risk free rate, smb, hml factors for the period 1983-2022



In [ ]:
ff_factors = pd.read_csv('FF-Factors-1983-2022.csv')

### List varibles in FF dataframe

In [ ]:
ff_factors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mktrf   480 non-null    float64
 1   smb     480 non-null    float64
 2   hml     480 non-null    float64
 3   rf      480 non-null    float64
 4   date    480 non-null    object 
dtypes: float64(4), object(1)
memory usage: 18.9+ KB


###Look at head and tail of dataframe

In [ ]:
ff_factors.head()

,mktrf,smb,hml,rf,date
0,0.0360,0.0273,-0.0075,0.0069,1983-01-31
1,0.0259,0.0327,0.0070,0.0062,1983-02-28
2,0.0282,0.0173,0.0202,0.0063,1983-03-31
3,0.0667,0.0050,0.0049,0.0071,1983-04-29
4,0.0052,0.0624,-0.0140,0.0069,1983-05-31


In [ ]:
ff_factors.tail()

,mktrf,smb,hml,rf,date
475,-0.0377,0.0137,0.0030,0.0019,2022-08-31
476,-0.0935,-0.0079,0.0006,0.0019,2022-09-30
477,0.0783,0.0009,0.0805,0.0023,2022-10-31
478,0.0460,-0.0340,0.0138,0.0029,2022-11-30
479,-0.0641,-0.0068,0.0132,0.0033,2022-12-30


## Create a function to process stock return data and calculate
## FF Regression coefficients for a stock ticker
### Steps:

1.   Create list of stocks with 60 months of complete stock return data from January 2018 to December 2022.
2.   Create function with passable parameter of the company's Ticker symbol (tic)

* Create dataframe for single company from main dataset
* Merge company's retrun data with Fama French data
* Drop missing values that rise from matching
* Run FF regression
* Store estimated coefficients in a Pandas dataframe

3. Use "Loop" function to estimate the regression for each ticker. Sort the datafile from the lowest value to the highest value of the variable "const", save the results to a file named "FF-Output-2018-2020.csv"




In [ ]:
# Step 1: Create list of stocks with 60 months of complete stock return data from January 2018 to December 2022.
ret_data1 = ret_data["TICKER"].value_counts().reset_index()
filter = ret_data1['count'] == 60
list = ret_data1[filter]['TICKER'].unique()
list

array(['PMO', 'AKR', 'NOM', ..., 'MGM', 'AMLP', 'HBB'], dtype=object)

In [ ]:
len(list)
#number of stocks with complete return data during 2018-2022

4966

In [ ]:
# Step 2: Create the function with passable paramneter of the company's Ticker symbol (tic)
def reg_coeff(tic):
  stock_data = ret_data[ret_data["TICKER"] == tic]
  stock_ff = pd.merge(stock_data, ff_factors, on='date', how='outer')
  stock_ff_final = stock_ff.dropna(axis=0,how='any')

  # Determine # of observations (should be 60)
  nobs=len(stock_ff_final.index)

  if nobs == 60:

# Regression for FF model
    y = stock_ff_final["RET"] - stock_ff_final["rf"]
    X = stock_ff_final[['mktrf' , 'smb' , 'hml']]
# Use statsmodels
    X = sm.add_constant(X) # adding a constant
    model = sm.OLS(y, X).fit()
    coeff = model.params
    temp_df=pd.DataFrame({tic:coeff})
    coeff_df=temp_df.transpose()
    return(temp_df)
  else:
    return("Less than 60 monthly stock returns for " + tic)

In [ ]:
# Step 3: Use "Loop" function to estimate the regression for each ticker. Sort the datafile from the lowest value to the highest value of the variable "const", save the results to a file named "FF-Output-2018-2020.csv"
d = {'index': ['const','mktrf','smb','hml']}
df = pd.DataFrame(data=d)
df

,index
0,const
1,mktrf
2,smb
3,hml


In [ ]:
for i in list:
    df1 = pd.DataFrame(reg_coeff(i)).reset_index()
    df = pd.merge(df1, df, on='index', how='outer')
print(df.transpose())

              0         1         2         3
index     const     mktrf       smb       hml
HBB   -0.004134   0.64078  0.630656  0.563957
AMLP  -0.000975    1.2864  0.689177  1.116333
MGM   -0.002151   1.87584  0.619888  0.911731
SCHP  -0.000571  0.178529 -0.040737 -0.091297
...         ...       ...       ...       ...
NSYS   0.028212  0.781928  0.401816 -0.288336
SPE   -0.001415   0.95496   0.22573  0.276773
NOM   -0.003931  0.271292  0.111672 -0.117633
AKR   -0.008921  1.187412  1.149877  0.920614
PMO   -0.000971  0.435629  0.088795 -0.108851

[4967 rows x 4 columns]


In [ ]:
df3 = df.transpose().iloc[1:]
df3.columns = ['const','mktrf','smb','hml']
df4 = df3.sort_values(by = ['const'], ascending= True)

In [ ]:
# Save the results in a csv file named "FF-Output-2018-2020.csv"
df4.to_csv('FF-Output-2018-2020.csv')

In [ ]:
# Top 3 stocks with highest estimated "alpha"
df4.tail(3)

,const,mktrf,smb,hml
UONE,0.255357,0.086765,10.840271,-0.370757
GME,0.336019,-3.79545,30.997736,5.526624
KOSS,0.338668,-4.422049,34.135444,5.502248


**COMPANY DESCRIPTION AND EXPLANATION FOR HIGH ESTIMATION OF ALPHA**

1. UONE: Urban One is a multimedia company primarily targeting African-American and urban audiences. During the COVID-19 pandemic, there was heightened awareness of social and racial issues, leading to increased demand for content catering to diverse audiences. Urban One's focus on this demographic may have positioned it well to benefit from this trend, resulting in higher estimated alpha during the crisis period.

2. GME: GameStop Corp. is an American video game, consumer electronics, and gaming merchandise retailer. The company is headquartered in Grapevine, Texas, and is the largest video game retailer worldwide. GME could have benefited from the rise in online gaming during stay-at-home restrictions.

3. KOSS: Koss Corporation is a company in Milwaukee, Wisconsin, US that manufactures audio equipment such as headphones. During the COVID-19 pandemic, there was a significant shift towards remote work and increased demand for home entertainment products. With people spending more time at home, the demand for high-quality headphones for remote work, virtual meetings, and entertainment purposes likely surged, benefiting companies like KOSS.

In [ ]:
# Top 3 stocks with lowest estimated "alpha"
df4.head(3)

,const,mktrf,smb,hml
INPX,-0.141599,0.911818,1.220058,0.426229
TNXP,-0.120614,1.381028,1.080373,-0.234476
VIVE,-0.107846,0.230479,1.411904,-0.386325


**COMPANY DESCRIPTION AND EXPLANATION FOR LOW ESTIMATION OF ALPHA**

1. Viveve Medical, Inc. (VIVE):

* Viveve Medical is a medical technology company that focuses on women's intimate health. During the COVID-19 pandemic, there might have been disruptions to elective medical procedures and a decrease in consumer spending on non-essential healthcare services. This could have negatively impacted Viveve's revenue and growth prospects, leading to a lower estimated alpha.

* Additionally, medical technology companies often require significant investment in research and development, which may have strained VIVE's profitability during a period of economic uncertainty and reduced investor appetite for speculative stocks.

2. Tonix Pharmaceuticals Holding Corp. (TNXP):

* Tonix Pharmaceuticals is a clinical-stage biopharmaceutical company focused on developing treatments for psychiatric and neurological conditions. Biopharmaceutical companies typically face high levels of uncertainty due to the lengthy and expensive drug development process, as well as regulatory hurdles.

* During the COVID-19 pandemic, there might have been delays in clinical trials, disruptions to supply chains, and increased competition for funding within the healthcare sector. These factors could have contributed to a lack of positive news or catalysts for TNXP's stock price, resulting in a lower estimated alpha.

3. Inpixon (INPX):

* Inpixon is a company that provides indoor positioning and data analytics solutions. The COVID-19 pandemic led to widespread closures of indoor spaces such as retail stores, offices, and entertainment venues, which are key markets for Inpixon's services.

* With reduced foot traffic and economic activity in indoor spaces, Inpixon may have experienced declining demand for its products and services, leading to lower revenue and profitability. This lack of growth opportunities and potential market contraction could have contributed to a lower estimated alpha for INPX during the crisis period.